In [1]:
import math
import numpy as np
from numpy import linalg as LA

In [2]:
EPS_ZEIDEL = 1e-9
ITERATIONS_ZEIDEL = 200000

In [3]:
def equalVector(a, b, eps):
    return math.sqrt(sum([(a[i] - b[i])**2 for i in range(len(a))])) < eps

In [4]:
def getLambdas(A):
    m = LA.eigvals(A)
    maxV = np.max(m)
    minV = np.min(m)
    return (maxV, minV)

In [5]:
def zeidelMethod(A, b):
    n = len(A)
    x = [100.0 for i in range(n)]
    xn = [.0 for i in range(n)]
    gotNAN = False
    curIterations = 0
    (ma, mi) = getLambdas(A)
    t = 1
    if (mi != 0):
        t = 2 / (ma + mi)
        A = t * A
        b = t * b
    print(f"Zeidel correction coefficient = {t}")
    while not equalVector(x, xn, EPS_ZEIDEL) and curIterations < ITERATIONS_ZEIDEL and not gotNAN:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            if np.isnan(xn[i]):
                print("Zeidel Got nan")
                gotNAN = True
                xn = [np.nan for i in range(n)]
                break
                
    print(f"Zeidel Iterations = {curIterations}. Max is {ITERATIONS_ZEIDEL} iterations.")
    return xn

In [6]:
def zeidelMethodRelax(A, b):
    n = len(A)
    x = [100.0 for i in range(n)]
    xn = [.0 for i in range(n)]
    Relax = 1.6
    gotNAN = False
    curIterations = 0
    (ma, mi) = getLambdas(A)
    t = 1
    if (mi != 0):
        t = 2 / (ma + mi)
        A = t * A
        b = t * b
    print(f"Zeidel correction coefficient = {t}")
    while not equalVector(x, xn, EPS_ZEIDEL) and curIterations < ITERATIONS_ZEIDEL and not gotNAN:
        x = np.copy(xn)
        curIterations += 1
        for i in range(n):
            s1 = - sum(A[i][j] * xn[j] for j in range(i)) / A[i][i]
            s2 = - sum(A[i][j] * x[j] for j in range(i + 1, n)) / A[i][i]
            xn[i] = (b[i] / A[i][i] + s1 + s2)
            xn[i] = Relax * xn[i] + (1 - Relax) * x[i]
            if np.isnan(xn[i]):
                print("Zeidel Relax Got nan")
                gotNAN = True
                xn = [np.nan for i in range(n)]
                break
            
    print(f"Zeidel Relax Iterations = {curIterations}. Max is {ITERATIONS_ZEIDEL} iterations.")
    return xn

Критерии остановки метода Зейделя - по близости двух последовательных приближений и ограничения максимального числа итераций (2000000). Вектор начального приближения заполняем нулями. Коэффициент релаксации - 1.6, так как именно такой коэффициент, исходя из экспериментов, ускоряет медленную сходимость, как в случае с матрицей Гильберта. Также мы пробовали коэффициент нижней релаксации (< 1), которая может дать сходимость, если ее нет, но пришли к выводу, что наиболее оптимальный результат получается при верхней релаксиции.